In [ ]:
# !pip install -Uq fastai

In [ ]:
#from optiver_features import generate_test_df
from fastai.tabular.all import *

In [ ]:
import cudf

In [ ]:
def wap_calc(df, bid_price, bid_size, ask_price, ask_size):
    return (df[bid_price]*df[ask_size]+df[ask_price]*df[bid_size])/(df[bid_size]+df[ask_size])

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def log_time(time_ids):
    return np.log(time_ids)

# # Calculate the realized volatility
# def realized_volatility(series):
#     return series.pipe(lambda x: np.sqrt(np.sum(x**2)))
#     #series=list(series)
#     #return cudf.sqrt(cudf.add(cudf.multiply(series,series)))
#     #return np.sqrt(np.sum(series*series))

In [ ]:
train_df = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_df['row_id'] = train_df['stock_id'].astype(str) + '-' + train_df['time_id'].astype(str)

### Generate Train Trade Features

In [ ]:
trade_train_df = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet")

In [ ]:
trade_train_df.head()

In [ ]:
trade_train_features = trade_train_df.groupby(['stock_id', 'time_id'], as_index=False).agg({'order_count':['sum'], 'size':['mean','sum','std'], 'price':['mean','std'], 'seconds_in_bucket':['mean','std']})
trade_train_features['row_id'] = trade_train_features['stock_id'].astype(str) + '-' + trade_train_features['time_id'].astype(str)
trade_train_features.columns = ['_'.join(col).strip('_') for col in trade_train_features.columns]
trade_train_features = trade_train_features.drop(['stock_id','time_id'], axis=1)

In [ ]:
train_df = train_df.merge(trade_train_features, how='left', on='row_id')

### Generate Train Book Features

In [ ]:
train_book_file_list = Path("../input/optiver-realized-volatility-prediction/book_train.parquet").ls()

In [ ]:
full_book_train_features = []
groupby_dict = {'bid_price1':['mean', 'std'], 
                'ask_price1':['mean', 'std'], 
                'bid_price2':['mean'], 
                'ask_price2':['mean'], 
                'bid_size1':['mean','sum','std', 'max'], 
                'bid_size2':['mean','sum','std', 'max'], 
                'ask_size1':['mean','sum','std', 'min', 'max'], 
                'ask_size2':['mean','sum','std', 'min', 'max'], 
                'wap1':['mean','sum','std', 'min', 'max'], 
                'wap2':['mean','sum','std', 'min', 'max'], 
                'log_return1':['mean', 'sum','std', 'min', 'max'], 
                'square_log_return1':['sum'], 
                'log_return2':['mean', 'sum','std', 'min', 'max'], 
                'square_log_return2':['sum'], 
                'wap_diff':['mean','sum','std', 'min', 'max'], 
                'seconds_in_bucket':['mean','std', 'max'], 
                'log_sib':['mean', 'std', 'max'],
                'int_sib':['mean', 'std', 'max'],
                'total_volume':['sum', 'mean', 'std', 'max', 'min'], 
                'price_spread':['sum', 'mean', 'std', 'max', 'min'],
                'price_spread2':['sum', 'mean', 'std', 'max', 'min'],
                'bid_spread':['sum', 'mean', 'std', 'max', 'min'],
                'ask_spread':['sum', 'mean', 'std', 'max', 'min'],
                'bid_ask_spread':['sum', 'mean', 'std', 'max', 'min'],
                'total_volume':['sum', 'mean', 'std', 'max', 'min']
               }
for fn in train_book_file_list:
    book_train_df = cudf.read_parquet(fn)
    book_train_df['stock_id'] = fn.name.split('=')[-1]
    book_train_df['wap1'] = wap_calc(book_train_df, 'bid_price1', 'bid_size1', 'ask_price1', 'ask_size1')
    book_train_df['wap2'] = wap_calc(book_train_df, 'bid_price2', 'bid_size2', 'ask_price2', 'ask_size2')
    book_train_df['log_return1'] = log_return(book_train_df['wap1'])
    book_train_df['log_return2'] = log_return(book_train_df['wap2'])
    book_train_df['wap_diff'] = book_train_df['wap1'] - book_train_df['wap2']
    book_train_df['square_log_return1'] = book_train_df['log_return1']**2
    book_train_df['square_log_return2'] = book_train_df['log_return2']**2
    # Calculate spread (https://www.kaggle.com/mayangrui/lgbm-ffnn)
    book_train_df['price_spread'] = (book_train_df['ask_price1'] - book_train_df['bid_price1']) / ((book_train_df['ask_price1'] + book_train_df['bid_price1']) / 2)
    book_train_df['price_spread2'] = (book_train_df['ask_price2'] - book_train_df['bid_price2']) / ((book_train_df['ask_price2'] + book_train_df['bid_price2']) / 2)
    book_train_df['bid_spread'] = book_train_df['bid_price1'] - book_train_df['bid_price2']
    book_train_df['ask_spread'] = book_train_df['ask_price1'] - book_train_df['ask_price2']
    book_train_df["bid_ask_spread"] = abs(book_train_df['bid_spread'] - book_train_df['ask_spread'])
    book_train_df['total_volume'] = (book_train_df['ask_size1'] + book_train_df['ask_size2']) + (book_train_df['bid_size1'] + book_train_df['bid_size2'])
    book_train_df['log_sib'] = log_time(book_train_df.seconds_in_bucket)
    book_train_df['int_sib'] = book_train_df.seconds_in_bucket//100
    book_train_features = book_train_df.groupby(by=['stock_id', 'time_id'], as_index=False).agg(groupby_dict)
    book_train_features['row_id'] = book_train_features['stock_id'].astype(str) + '-' + book_train_features['time_id'].astype(str)
    book_train_features.columns = ['_'.join(col).strip('_') for col in book_train_features.columns]
    book_train_features = book_train_features.drop(['stock_id','time_id'], axis=1)
    full_book_train_features.append(book_train_features)

final_book_train_features = cudf.concat(full_book_train_features)
final_book_train_features = final_book_train_features.reset_index().rename({'index':'timeval'}, axis=1)
final_book_train_features.head()

final_book_train_features = final_book_train_features.to_pandas()

In [ ]:
final_book_train_features['realized_volatility1'] = final_book_train_features.square_log_return1_sum**(1/2)
final_book_train_features['realized_volatility2'] = final_book_train_features.square_log_return2_sum**(1/2)

In [ ]:
train_df = train_df.merge(final_book_train_features, how='left', on='row_id')

In [ ]:
train_df['log_time_id'] = log_time(train_df.time_id)
train_df['int_time_id0'] = train_df.time_id//10000
train_df['int_time_id1'] = train_df.time_id//1000
train_df['int_time_id2'] = train_df.time_id//100

In [ ]:
train_df.columns

In [ ]:
cont_columns, cat_columns = cont_cat_split(train_df, dep_var='target')

In [ ]:
cat_columns

In [ ]:
cat_columns.remove('row_id')
cont_columns.remove('stock_id')
cat_columns.append('stock_id')

In [ ]:
cat_columns

### Define splits

In [ ]:
splits = RandomSplitter()(train_df)

### Categorical vs Continuous

In [ ]:
to = TabularPandas(train_df, procs=[FillMissing(FillStrategy.mode), Normalize, Categorify], cat_names=cat_columns, cont_names=cont_columns, y_names='target', y_block=RegressionBlock(), reduce_memory=True, splits=splits)

In [ ]:
dls = to.dataloaders(bs=2048)

### Create Tabular Learner

In [ ]:
def rmspe(y_pred, y_true):
    return (torch.sqrt(torch.mean(torch.square((y_true - y_pred) / y_true))))

In [ ]:
max_y = train_df.target.max()*2

In [ ]:
tc = tabular_config(embed_p=0.6, ps=0.01)

In [ ]:
#learn = tabular_learner(dls, layers=[4096, 2048, 1024], metrics=[rmspe], loss_func=L1LossFlat(), y_range=(0, max_y))
#learn = tabular_learner(dls, layers=[1024,1024,512], metrics=[rmspe], loss_func=L1LossFlat(reduction='sum'), config=tc, y_range=(0, max_y))
learn = tabular_learner(dls, layers=[4096,2048,1024], loss_func=rmspe, config=tc, y_range=(0, max_y), opt_func=ranger)
#learn = tabular_learner(dls, layers=[1024,2048,4096], loss_func=rmspe, config=tc, y_range=(0, max_y), opt_func=ranger)

In [ ]:
learn = learn.to_fp32()

### Train Model

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(25, 1e-3)

In [ ]:
preds=learn.get_preds();preds

In [ ]:
test_df = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')

In [ ]:
trade_test_df = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet")

In [ ]:
trade_test_features = trade_test_df.groupby(['stock_id','time_id'], as_index=False).agg({'order_count':['sum'], 'size':['mean','sum','std'], 'price':['mean','sum','std'], 'seconds_in_bucket':['mean','std']})
trade_test_features['row_id'] = trade_test_features['stock_id'].astype(str) + '-' + trade_test_features['time_id'].astype(str)
trade_test_features.columns = ['_'.join(col).strip('_') for col in trade_test_features.columns]
trade_test_features = trade_test_features.drop(['stock_id','time_id'], axis=1)

In [ ]:
test_df = test_df.merge(trade_test_features, how='left', on='row_id')

### Test Book Features

In [ ]:
test_book_file_list = Path("../input/optiver-realized-volatility-prediction/book_test.parquet").ls()

In [ ]:
full_book_test_features = []
for fn in test_book_file_list:
    book_test_df = cudf.read_parquet(fn)
    book_test_df['stock_id'] = fn.name.split('=')[-1]
    book_test_df['wap1'] = wap_calc(book_test_df, 'bid_price1', 'bid_size1', 'ask_price1', 'ask_size1')
    book_test_df['wap2'] = wap_calc(book_test_df, 'bid_price2', 'bid_size2', 'ask_price2', 'ask_size2')
    book_test_df['log_return1'] = log_return(book_test_df['wap1'])
    book_test_df['log_return2'] = log_return(book_test_df['wap2'])
    book_test_df['wap_diff'] = book_test_df['wap1'] - book_test_df['wap2']
    book_test_df['square_log_return1'] = book_test_df['log_return1']**2
    book_test_df['square_log_return2'] = book_test_df['log_return2']**2
    book_test_df['total_volume'] = book_test_df['ask_size1']+book_test_df['ask_size2']+book_test_df['bid_size1']+book_test_df['bid_size2']
    # Calculate spread (https://www.kaggle.com/mayangrui/lgbm-ffnn)
    book_test_df['price_spread'] = (book_test_df['ask_price1'] - book_test_df['bid_price1']) / ((book_test_df['ask_price1'] + book_test_df['bid_price1']) / 2)
    book_test_df['price_spread2'] = (book_test_df['ask_price2'] - book_test_df['bid_price2']) / ((book_test_df['ask_price2'] + book_test_df['bid_price2']) / 2)
    book_test_df['bid_spread'] = book_test_df['bid_price1'] - book_test_df['bid_price2']
    book_test_df['ask_spread'] = book_test_df['ask_price1'] - book_test_df['ask_price2']
    book_test_df["bid_ask_spread"] = abs(book_test_df['bid_spread'] - book_test_df['ask_spread'])
    book_test_df['total_volume'] = (book_test_df['ask_size1'] + book_test_df['ask_size2']) + (book_test_df['bid_size1'] + book_test_df['bid_size2'])
    book_test_df['log_sib'] = log_time(book_test_df.seconds_in_bucket)
    book_test_df['int_sib'] = book_test_df.seconds_in_bucket//100
    book_test_features = book_test_df.groupby(by=['stock_id', 'time_id'], as_index=False).agg(groupby_dict)
    book_test_features['row_id'] = book_test_features['stock_id'].astype(str) + '-' + book_test_features['time_id'].astype(str)
    book_test_features.columns = ['_'.join(col).strip('_') for col in book_test_features.columns]
    book_test_features = book_test_features.drop(['stock_id','time_id'], axis=1)
    full_book_test_features.append(book_test_features)

final_book_test_features = cudf.concat(full_book_test_features)
final_book_test_features = final_book_test_features.reset_index().rename({'index':'timeval'}, axis=1)

final_book_test_features = final_book_test_features.to_pandas()

In [ ]:
final_book_test_features

In [ ]:
final_book_test_features['realized_volatility1'] = final_book_test_features.square_log_return1_sum**(1/2)
final_book_test_features['realized_volatility2'] = final_book_test_features.square_log_return2_sum**(1/2)

In [ ]:
test_df = test_df.merge(final_book_test_features, how='left', on='row_id')

In [ ]:
test_df['log_time_id'] = log_time(test_df.time_id)
test_df['int_time_id0'] = test_df.time_id//10000
test_df['int_time_id1'] = test_df.time_id//1000
test_df['int_time_id2'] = test_df.time_id//100

In [ ]:
test_df = test_df.fillna(method='ffill')

### Create Test Preds

In [ ]:
dl = learn.dls.test_dl(test_df)

In [ ]:
test_pred = learn.get_preds(dl=dl)

In [ ]:
test_pred[0].squeeze()

In [ ]:
#submit.target = test_pred[0].squeeze().tolist()
test_df['target'] = test_pred[0].squeeze().tolist()
test_df

In [ ]:
submit = test_df[['row_id', 'target']].copy()

In [ ]:
submit.to_csv('submission.csv',index=False)

In [ ]:
submit